# Model Test Bench

In [1]:
from keras.models import load_model
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import butter
from hampel import hampel
from tqdm.notebook import trange, tqdm
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
import h5py

In [2]:
# df = pd.read_excel('Myotron_Controll_codes/sim_data/prosup_raw_200s_labeled.xlsx')
df = pd.read_csv('prepared_data/restprosup_nina_raw.csv')
df

,t[s],emg1,emg2,emg3,emg4,emg5,emg6,emg7,emg8,labels
0,859880,4.020336e-06,3.576222e-07,-1.224552e-06,9.989841e-07,-4.281166e-07,-7.940087e-07,-0.000012,-0.000007,1
1,859881,3.349212e-06,-1.458975e-07,-1.560250e-06,4.956104e-07,1.211436e-07,-2.304278e-06,-0.000011,-0.000007,1
2,859882,3.349484e-06,8.612258e-07,1.182418e-07,1.502881e-06,2.178858e-08,-2.303903e-06,-0.000011,-0.000006,1
3,859883,4.356853e-06,8.612530e-07,2.300282e-06,3.852945e-06,-2.602581e-07,-1.296432e-06,-0.000011,-0.000003,1
4,859884,6.035617e-06,1.898822e-07,1.628885e-06,5.867310e-06,-3.822630e-07,4.673569e-08,-0.000012,-0.000002,1
...,...,...,...,...,...,...,...,...,...,...
412819,1185326,1.737583e-06,-1.536634e-07,2.693880e-07,9.109114e-06,-2.774911e-06,3.383996e-06,-0.000005,0.000004,0
412820,1185327,3.214152e-06,2.367024e-07,1.612182e-06,9.948470e-06,-4.117855e-06,4.558857e-06,-0.000007,0.000004,0
412821,1185328,1.010438e-06,-2.138706e-07,1.780031e-06,9.780731e-06,-2.103816e-06,5.565867e-06,-0.000008,0.000006,0
412822,1185329,-1.511917e-06,-2.677504e-06,1.015388e-07,9.948690e-06,1.253018e-06,5.565783e-06,-0.000007,0.000006,0


In [3]:
def make_windowed(data,win_len,win_inc):
    nb = data.shape[0]
    X = []
    inc = 0
    i = 0
    while True:
        if(i==1):
            inc=win_inc
        if(data[i+win_inc:i+win_len+win_inc].shape[0]!=win_len):
            break
#         print(data[i+win_inc:i+win_len+win_inc].shape,' ',i)
        X.append(data[i+win_inc:i+win_len+win_inc])
        i=i+1
    # np.save("prepared_data/X", X)
    return np.array(X)
def make_windowed_labels(y_label,win_len,win_inc):
    y_windowed = make_windowed(y_label,win_len,win_inc)
    Y = []
    for x in y_windowed:
        Y.append(np.bincount(x).argmax())
    return np.array(Y)
def one_hot(data,n_variables):
    n = len(data)
    Y = np.zeros((n,n_variables))
    for i in range(n):
        t = data[i]
        Y[i][t] = 1
    return Y
def normalize_whole(df):
    emg_labels = ['emg1','emg2','emg3','emg4','emg5','emg6','emg7','emg8']
    emg_df = df[emg_labels]
    scaler = StandardScaler(with_mean=True,
                            with_std=True,
                            copy=False).fit(emg_df)
#     print(scaler.mean_)
#     print(scaler.scale_)
#     print(scaler.n_features_in_)
#     print(scaler.n_samples_seen_)
#     print(scaler.var_)
    emg_df = scaler.transform(emg_df)
    df[emg_labels] = emg_df
    return df, scaler

def normalize_whole_minmax(df):
    emg_labels = ['emg1','emg2','emg3','emg4','emg5','emg6','emg7','emg8']
    emg_df = df[emg_labels]
    scaler = MinMaxScaler()
    scaler.fit(emg_df)
#     print(scaler.mean_)
#     print(scaler.scale_)
#     print(scaler.n_features_in_)
#     print(scaler.n_samples_seen_)
#     print(scaler.var_)
    emg_df = scaler.transform(emg_df)
    df[emg_labels] = emg_df
    return df, scaler

In [4]:
# df,scaler = normalize_whole(df) # Only for Nina Prep
df,scaler = normalize_whole_minmax(df)
import joblib
joblib.dump(scaler, 'prepared_data/restprosup_nina_minmax_scaler.pkl') 

['prepared_data/restprosup_nina_minmax_scaler.pkl']

In [5]:
emg_labels = ['emg1','emg2','emg3','emg4','emg5','emg6','emg7','emg8']

# rein_emg_labels = ['emg8','emg1','emg2','emg3','emg4','emg5','emg6','emg7']
# all_labels = ['t[s]']+ rein_emg_labels + ['labels']
# rdf = df.reindex(columns=all_labels)
X = make_windowed(np.array(df[emg_labels]),1000,200)

In [6]:
X.shape

(411625, 1000, 8)

In [7]:
y_label = np.array(df['labels'])
Y = make_windowed_labels(y_label,1000,200)
y = one_hot(Y,3)
y.shape

(411625, 3)

In [8]:
del df

In [ ]:
np.save('prepared_data/X',X)
np.save('prepared_data/y',y)

In [ ]:
del X, Y, y

# Model Test

In [ ]:
def classify(data,model):
    data = np.reshape(data,(1,win_len,n_channels))
    soft = model.predict(data,batch_size=1)
    clf = np.argmax(np.array(soft))
    return clf

In [ ]:
model_file = 'models/restprosup_nina_model_250'
model = load_model(model_file)

In [ ]:
model.evaluate(X,y,batch_size=64)

# InceptionTime Trainer

In [ ]:
X = np.load('prepared_data/X.npy')
y = np.load('prepared_data/y.npy')

In [10]:
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,shuffle=True)

In [ ]:
X_train, X_test, y_train, y_test = X[:500000], X[500000:], y[:500000], y[500000:]

In [ ]:
del X,y

In [ ]:
y_true = []
for d in y_test:
    idx = list(d).index(1)
    y_true.append(idx)

In [ ]:
print('X Train Shape =',X_train.shape)
print('Y Train Shape =',y_train.shape)
print('X Test Shape =',X_test.shape)
print('Y Test Shape =',y_test.shape)

In [11]:
ds = [X_train, X_test, y_train, y_test]
ds_name = ['X_train', 'X_test', 'y_train', 'y_test']
for i in range(len(ds_name)):
    np.save('prepared_data/'+ds_name[i],ds[i])

In [ ]:
X_train = np.load('prepared_data/X_train.npy')
X_test = np.load('prepared_data/X_test.npy')
y_train = np.load('prepared_data/y_train.npy')
y_test = np.load('prepared_data/y_test.npy')

## Build New Model

In [ ]:
import numpy as np
import sklearn
import pandas as pd
import sys
import subprocess
import tensorflow as tf
from InceptionTime.classifiers.inception import Classifier_INCEPTION

In [ ]:
 clf = Classifier_INCEPTION('', (X_train.shape[1],X_train.shape[2]),nb_classes=3,
                            verbose=True,batch_size=64,nb_epochs=25,nb_filters=16,
                            depth=1, kernel_size=11)

In [ ]:
clf.model = load_model('best_model.hdf5') # if continuing Training
# clf.model = load_model('models/restprosup_fazil_model_250')

In [ ]:
df_metrics = clf.fit(X_train, y_train, X_test, y_test, y_true,plot_test_acc=True)

In [ ]:
from keras.models import load_model
best_model = load_model('best_model.hdf5')
best_model.save('models/restprosup_nina_model_{}'.format(1000))